In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib as plt
import matplotlib.pyplot as plt
import sklearn.preprocessing as sp
%matplotlib inline
sns.set_style('whitegrid')
sns.set_context("poster")
sns.set_context("talk", 1.0, {"lines.linewidth": 3})

In [2]:
#パターン14のデータの取得
df14_2 = pd.read_csv("./取得データセット/受信機2/nkmr2-dataset14.csv")
df14_4 = pd.read_csv("./取得データセット/受信機4/nkmr4-dataset14.csv")

#それぞれのカラムの指定
df14_2.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x_label"]
df14_4.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x_label"]
df14 = df14_2.append(df14_4)

#同期ビーコンのid
set_uuid = "1000000007ae1001b000001c4d8dffff"

#同期用ビーコンの値を抜く
#df14 = df14[df14.UUIDs != "1000000007ae1001b000001c4d8dffff"]

#座標ごとに分ける
df14x1 = df14[df14.x_label == 100]
df14x2 = df14[df14.x_label == 200]
df14x3 = df14[df14.x_label == 300]
df14x4 = df14[df14.x_label == 400]
df14x5 = df14[df14.x_label == 500]

df_list = [df14x1,df14x2,df14x3,df14x4,df14x5]


#locateを作成
df14["locate"] = df14.device_name.apply(lambda x: 2 if x == "nkmr-2" else 4)
df14["locate"] = df14.locate.astype(int)

### 時刻合わせ

In [3]:
#　オブジェクト型の目的変数を数値情報に変換
import sklearn.preprocessing as sp

# 前処理用のインスタンスを宣言
le = sp.LabelEncoder()

def set_time(data,set_uuid):
    """
    時刻合わせを行う
    第一引数はdataframe
    第２引数は時刻合わせを行うUUID
    """
    # 時刻合わせ用のビーコンを1秒間隔に設定
    set_time_df = data[data.UUIDs == set_uuid].drop_duplicates(['Minor'],keep = "first")
    # 時刻合わせ用ビーコン以外を定義
    df = data[data.UUIDs != set_uuid]
    # データを結合してインデックスでソート
    df = df.append(set_time_df).sort_index(ascending=True)
    # 時刻合わせ用ビーコンを取得した行にtimeカラムを設定
    df["time"] = df["Minor"].where(df["UUIDs"] == set_uuid)
    # 時刻合わせを行い欠損値がある部分は削除
    df = df.fillna(method="ffill").dropna()
    # int型に変換
    df["time"] = df["time"].astype(int)
    # 時刻合わせ用のビーコンデータ以外
    df = df[df["UUIDs"] != set_uuid]
    
    return df

# 前処理を各データフレームに行う
for (i,df) in enumerate(df_list):
    df_list[i] = set_time(df,set_uuid)
    # device_nameを変換
    df_list[i]["device"] = le.fit_transform(df_list[i][df_list[i].columns[7]]) + 1

In [4]:
df_list[1]

,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x_label,time,device
916,10000000aaaa41441111110000000000,2,38,1.612630,near,-78,-88,nkmr-2,2017/12/15 15:35:18,200,62,1
918,10000000aaaa41441111110000000000,2,38,1.537381,near,-78,-87,nkmr-2,2017/12/15 15:35:18,200,62,1
921,10000000aaaa41441111110000000000,2,38,1.537381,near,-78,-87,nkmr-2,2017/12/15 15:35:19,200,62,1
935,10000000aaaa41441111110000000000,2,38,1.774360,near,-78,-90,nkmr-2,2017/12/15 15:35:20,200,63,1
937,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,nkmr-2,2017/12/15 15:35:20,200,63,1
951,10000000aaaa41441111110000000000,2,38,2.047870,near,-78,-93,nkmr-2,2017/12/15 15:35:21,200,64,1
958,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,nkmr-2,2017/12/15 15:35:21,200,64,1
959,10000000aaaa41441111110000000000,2,38,1.861210,near,-78,-91,nkmr-2,2017/12/15 15:35:21,200,64,1
985,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,nkmr-2,2017/12/15 15:35:23,200,66,1
988,10000000aaaa41441111110000000000,2,38,1.861210,near,-78,-91,nkmr-2,2017/12/15 15:35:23,200,67,1


### 直接波と反射波を識別する

In [5]:
# KMeansライブラリをインポート
from sklearn.cluster import KMeans

In [6]:
def make_cluster(df_list,colum,*,N_CLUSTERS=2):
    """
          反射波と入射波を判別するクラスタを作成し描写を行う
          第1引数(地点毎のデータフレームが入ったリスト)
          第２引数(クラスタリングを行いたいカラムのリスト)
          第３引数(クラスタ数)
          データフレーム型でクラスタリング結果を返す
    """

    # 空のデータフレームを用意
    df_cluster = pd.DataFrame()
    concat_list = []
    #座標ごとに
    for df in df_list: 
        #受信機ごとに
        for dev in df.device_name.unique():
            #受信機ごとに分けたdataframe
            df_dev = df[df.device_name == dev]
            # 指定されたカラムを利用してクラスタリング
            pred = KMeans(n_clusters=N_CLUSTERS).fit_predict(np.array(df_dev[colum]))
            # clusterカラムを作成し予測結果を代入
            df_dev.loc[:,("cluster")] = pred
            # RSSI値の平均値を比較して入射波と反射波の判定を行う
            if df_dev[df_dev["cluster"] == 0]["RSSI"].mean() > df_dev[df_dev["cluster"] == 1]["RSSI"].mean():
                pass
            else:
                df_dev[df_dev["cluster"] == 0]["cluster"] = 3
                df_dev[df_dev["cluster"] == 1]["cluster"] = 0
                df_dev[df_dev["cluster"] == 3]["cluster"] = 1
            # 予測結果を格納
            df_cluster = pd.concat([df_cluster,df_dev],ignore_index=True)
        concat_list.append(df_cluster)
        # 空のデータフレームを用意
        df_cluster = pd.DataFrame()

    return concat_list

In [7]:
df_list = make_cluster(df_list,["RSSI","x_label"]) 

/Users/tomanan/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/tomanan/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/tomanan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [8]:
df_list[0].time.unique()

array([   1,    2,    3,    4,    5,    6,    7,    9,   10,   11,   12,
         13,   14,   15,   16,   17,   18,   19,   20,   21,   22,   23,
         24,   25,   26,   27,   28,   29,   30,   31,   32,   33,   34,
         35,   36,   37,   38,   39,   40,   41,   42,   43,   44,   45,
         46,   47,   48,   49,   50,   51,   52,   53,   54,   55,   56,
         57,   58,   59,  509,  510,  511,  512,  513,  514,  515,  516,
        517,  518,  519,  520,  521,  522,  523,  524,  525,  526,  527,
        528,  529,  530,  531,  532,  533,  534,  535,  536,  537,  538,
        539,  976,  977,  978,  979,  980,  981,  982,  983,  984,  985,
        986,  987,  988,  989,  990,  991,  992,  993,  994,  995,  996,
        997,  998,  999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007,
       1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018,
       1019,    8,  480,  482,  483,  484,  485,  486,  487,  488,  489,
        491,  496,  505,  506,  507,  960,  963,  9

In [9]:
def max_value(df_list):
    # timeのmax値を計算
    for df in df_list:
        max_value = 0
        if max_value < df["time"].max():
            max_value = df["time"].max()
    return max_value

def get_mean(df,colum_name,locate,time_max):    
    """
    処理時間かかります。
    時刻毎の中央値をとる
    第一引数、データフレーム
    第２引数、作成したいカラム名
    """
    # 空のデータフレームとリストを用意
    Direct_list = []
    Reflect_list = []
    mid_df = pd.DataFrame()
    
    # timeのユニーク数だけ
    for time_num in range(time_max): 
        # 受信機の数だけ
        for i in range(len(df_list[0]["device"].unique())): 
            # timeラベルが同じ値に対して平均値をとる
            Direct = df[(df["time"] == time_num) & (df["device"] == (i+1)) & (df["cluster"] == 0)]["RSSI"].mean()
            Reflect = df[(df["time"] == time_num) & (df["device"] == (i+1)) & (df["cluster"] == 1)]["RSSI"].mean()
            # 平均値をリストに追加する
            Direct_list.append(Direct)
            Reflect_list.append(Reflect)
        # 平均値を追加したリストに(時間、地点、クラスタ番号を追加)
        Direct_list.extend([time_num,locate,0])
        Reflect_list.extend([time_num,locate,1])
        # データフレームに各要素を追加
        mid_df = mid_df.append(pd.Series(Direct_list),ignore_index=True)
        mid_df = mid_df.append(pd.Series(Reflect_list),ignore_index=True)
        # 平均値のリストを初期化
        Direct_list = []
        Reflect_list = []
    # カラム名を定義
    mid_df.columns = [colum_name]
    
    return mid_df

#前処理を各データフレームと地点毎に行う
train_df = pd.DataFrame()
column_name = ["dev1","dev2","time","xlabel","cluster"]
time_max = max_value(df_list)
for df in df_list:
    xlabel = df["x_label"][0]
    mid_df = get_mean(df,column_name,(xlabel),time_max)
    train_df = pd.concat([train_df,mid_df])

In [34]:
train_df["dev1"]

0             NaN
1             NaN
2             NaN
3      -90.000000
4             NaN
5      -89.000000
6             NaN
7      -90.000000
8             NaN
9      -89.000000
10            NaN
11            NaN
12            NaN
13     -89.500000
14            NaN
15     -89.500000
16            NaN
17     -90.000000
18            NaN
19     -89.000000
20            NaN
21     -89.000000
22            NaN
23     -90.000000
24            NaN
25     -89.000000
26            NaN
27            NaN
28            NaN
29     -90.000000
          ...    
2488   -90.000000
2489   -88.000000
2490   -91.250000
2491   -87.500000
2492   -89.000000
2493          NaN
2494   -93.000000
2495   -87.000000
2496   -91.000000
2497   -87.500000
2498   -90.800000
2499   -88.000000
2500   -89.000000
2501   -87.000000
2502   -89.000000
2503   -88.000000
2504   -92.500000
2505          NaN
2506   -90.000000
2507   -87.000000
2508   -93.000000
2509   -88.000000
2510   -90.333333
2511   -88.000000
2512   -91

In [11]:
direct = train_df[train_df.iloc[:,4] == 0].dropna(thresh=4).interpolate().dropna()
reflect = train_df[train_df.iloc[:,4] == 1].dropna(thresh=4).interpolate().dropna()
train = pd.concat([direct,reflect])
train

,dev1,dev2,time,xlabel,cluster
50,-93.000000,-84.800000,25.0,100.0,0.0
52,-92.954545,-85.000000,26.0,100.0,0.0
54,-92.909091,-85.333333,27.0,100.0,0.0
56,-92.863636,-85.000000,28.0,100.0,0.0
58,-92.818182,-85.400000,29.0,100.0,0.0
60,-92.772727,-85.000000,30.0,100.0,0.0
62,-92.727273,-85.333333,31.0,100.0,0.0
64,-92.681818,-84.500000,32.0,100.0,0.0
66,-92.636364,-85.333333,33.0,100.0,0.0
68,-92.590909,-85.666667,34.0,100.0,0.0


### ランダムフォレスト

In [12]:
from sklearn.utils import shuffle

# 学習用データセットを用意
train_x = np.array(train[["dev1","dev2","cluster"]])
train_y = np.array(train["xlabel"])
train_x,train_y = shuffle(train_x,train_y)

In [13]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=5)
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

,pred,train_y
0,299.607456,300.0
1,343.229176,300.0
2,151.938004,100.0
3,299.607456,300.0
4,500.000000,500.0
5,282.397936,200.0
6,247.699705,200.0
7,353.240911,400.0
8,399.950495,400.0
9,212.720474,300.0


In [14]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

誤差の平均値:30.630827078692217


In [15]:
abs(hoge.pred - hoge.train_y)

0         0.392544
1        43.229176
2        51.938004
3         0.392544
4         0.000000
5        82.397936
6        47.699705
7        46.759089
8         0.049505
9        87.279526
10      123.482883
11        1.392342
12        0.477966
13       51.938004
14       56.240817
15        1.494333
16        1.392342
17       29.495048
18       39.810541
19       34.128414
20       46.759089
21       53.240911
22       17.602064
23        1.494333
24       18.509094
25        0.477966
26        0.658715
27        0.477966
28       70.504952
29        0.049505
           ...    
2100     35.933136
2101     51.938004
2102     58.135120
2103     13.325485
2104      0.392544
2105     48.061996
2106      0.477966
2107    252.300295
2108     29.495048
2109      0.049505
2110     29.495048
2111     47.699705
2112     17.602064
2113      7.668758
2114      4.183324
2115     11.133328
2116     10.175137
2117      0.477966
2118     35.933136
2119     17.602064
2120      5.685431
2121     14.

### 線形回帰

In [16]:
# sklearn.linear_model.LinearRegression クラスを読み込み
from sklearn import linear_model

model = linear_model.LinearRegression() 
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

,pred,train_y
0,321.132832,300.0
1,307.205974,300.0
2,292.286850,100.0
3,279.974267,300.0
4,395.788593,500.0
5,321.806000,200.0
6,271.230407,200.0
7,257.841146,400.0
8,430.350063,400.0
9,268.356524,300.0


In [17]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

誤差の平均値:80.67069273076791


### ロジスティック回帰

In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

,pred,train_y
0,400.0,300.0
1,400.0,300.0
2,200.0,100.0
3,300.0,300.0
4,400.0,500.0
5,300.0,200.0
6,200.0,200.0
7,200.0,400.0
8,400.0,400.0
9,200.0,300.0


In [19]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

誤差の平均値:71.22065727699531


### アンサンブル回帰のバギング

In [20]:
from sklearn import ensemble,tree

model = ensemble.BaggingRegressor(tree.DecisionTreeRegressor(max_depth = 5))
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

,pred,train_y
0,290.316888,300.0
1,343.714962,300.0
2,149.726800,100.0
3,290.316888,300.0
4,500.000000,500.0
5,288.814693,200.0
6,243.246630,200.0
7,350.951984,400.0
8,399.830302,400.0
9,203.732007,300.0


In [21]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

誤差の平均値:31.02941850292862


In [22]:
tree.DecisionTreeRegressor()

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### Lasso回帰

In [23]:
model = linear_model.Lasso() 
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

,pred,train_y
0,316.952174,300.0
1,304.722036,300.0
2,292.476376,100.0
3,276.267391,300.0
4,390.193706,500.0
5,322.194647,200.0
6,273.354366,200.0
7,255.900694,400.0
8,432.003141,400.0
9,268.077469,300.0


In [24]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

誤差の平均値:81.22618886837377


### svm

In [25]:
from sklearn import svm
model = svm.SVC() 
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

,pred,train_y
0,300.0,300.0
1,300.0,300.0
2,200.0,100.0
3,300.0,300.0
4,500.0,500.0
5,300.0,200.0
6,200.0,200.0
7,400.0,400.0
8,400.0,400.0
9,200.0,300.0


In [26]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

誤差の平均値:14.976525821596244


In [27]:
import xgboost as xgb
model = xgb.XGBRegressor()
reg = model.fit(train_x,train_y)
pred = reg.predict(train_x)

hoge = pd.concat([pd.Series(pred),pd.Series(train_y)],axis=1)
hoge.columns = ["pred","train_y"]
hoge

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
ave = abs(hoge.pred - hoge.train_y).mean()
print("誤差の平均値:%s" % ave)

In [38]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.1.final.0
python-bits: 64
OS: Darwin
OS-release: 17.2.0
machine: x86_64
processor: i386
byteorder: little
LC_ALL: None
LANG: ja_JP.UTF-8
LOCALE: ja_JP.UTF-8

pandas: 0.20.1
pytest: 3.0.7
pip: 9.0.1
setuptools: 27.2.0
Cython: 0.25.2
numpy: 1.12.1
scipy: 1.0.0
xarray: None
IPython: 5.3.0
sphinx: 1.5.6
patsy: 0.4.1
dateutil: 2.6.0
pytz: 2017.2
blosc: None
bottleneck: 1.2.1
tables: 3.3.0
numexpr: 2.6.4
feather: None
matplotlib: 2.0.2
openpyxl: 2.4.7
xlrd: 1.0.0
xlwt: 1.2.0
xlsxwriter: 0.9.6
lxml: 3.7.3
bs4: 4.6.0
html5lib: 0.999
sqlalchemy: 1.1.9
pymysql: None
psycopg2: None
jinja2: 2.9.6
s3fs: None
pandas_gbq: None
pandas_datareader: None
